In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import warnings


In [2]:
# Load the data from the primary sheet
data = pd.read_excel('ELB-Sales-Data.xlsx', sheet_name='Elbrit Sales Log')

# Display the first few rows of the data to understand its structure and identify relevant columns
data.head()


,Sr,ID,Customer,Primary Sales,Cancelled,Is added to primary log,Against Expiry,Breakage,Claim Offer,Quantity,...,Free Item,HQ,Sales Team,Return for Reason,Sales Return,Sales Invoice Name,Rate Difference,Item Name,Free Item Value,Date
0,9354,SL-10-24-268441,SESHABALAJEE MEDISOLUTIONS PVT LTD,135.08,1,1,135.08,0.0,0.0,-4,...,0,HQ-Vizag,Elbrit AP,Expired,0.0,SINV-24-01509,0.0,TELBRIT 20,0.0,2024-09-23
1,9355,SL-10-24-268440,SESHABALAJEE MEDISOLUTIONS PVT LTD,413.80,1,1,413.80,0.0,0.0,-4,...,0,HQ-Vizag,Elbrit AP,Expired,0.0,SINV-24-01509,0.0,DAFAX M 5,0.0,2024-09-23
2,9356,SL-10-24-268439,SESHABALAJEE MEDISOLUTIONS PVT LTD,210.38,1,1,210.38,0.0,0.0,-2,...,0,HQ-Vizag,Elbrit AP,Expired,0.0,SINV-24-01509,0.0,TELBRIT AM,0.0,2024-09-23
3,9357,SL-10-24-268438,SESHABALAJEE MEDISOLUTIONS PVT LTD,105.19,1,1,105.19,0.0,0.0,-1,...,0,HQ-Vizag,Elbrit AP,Expired,0.0,SINV-24-01509,0.0,TELBRIT H,0.0,2024-09-23
4,9358,SL-10-24-268437,SESHABALAJEE MEDISOLUTIONS PVT LTD,978.30,1,1,978.30,0.0,0.0,-9,...,0,HQ-Vizag,Elbrit AP,Expired,0.0,SINV-24-01509,0.0,TRIGLIMIBRIT 1.3,0.0,2024-09-23


In [3]:
data.columns

Index(['Sr', 'ID', 'Customer', 'Primary Sales', 'Cancelled',
       'Is added to primary log', 'Against Expiry', 'Breakage', 'Claim Offer',
       'Quantity', 'Net Rate', 'Free Item', 'HQ', 'Sales Team',
       'Return for Reason', 'Sales Return', 'Sales Invoice Name',
       'Rate Difference', 'Item Name', 'Free Item Value', 'Date'],
      dtype='object')

### 1. What was the highest-selling product in September?
---

In [4]:
# Convert the 'Date' column to datetime for filtering

data['Date'] = pd.to_datetime(data['Date'])

# Filter data for September 2024

september_data = data[(data['Date'].dt.month == 9) & (data['Date'].dt.year == 2024)]

# Group the September data by 'Item Name' and sum the 'Primary Sales'

highest_selling_product_sept = september_data.groupby('Item Name')['Primary Sales'].sum()

highest_selling_product_sept


Item Name
ACEBRIT MR         18514.80
ACEBRIT P          63959.76
AMOXIBRIT 625     329224.74
ARNIBLOC 100      162966.70
ARNIBLOC 50       381422.54
                    ...    
VEINEX            278814.15
VILZATO 100 OD     20047.23
VILZATO 50        249651.99
VILZATO M 1000      7456.80
VILZATO M 500     429683.93
Name: Primary Sales, Length: 186, dtype: float64

In [5]:
# Identify the product with the highest sales

highest_selling_product_sept_name = highest_selling_product_sept.idxmax()

highest_selling_product_sept_value = highest_selling_product_sept.max()

highest_selling_product_sept_name,highest_selling_product_sept_value

('NEURONZ D', np.float64(2308078.58))

### 2. Which product had the highest sales for the "CND Chennai" sales team in May?
---

In [6]:
# 2. Highest sales by "CND Chennai" sales team in May

may_data = data[(data['Date'].dt.month == 5) & (data['Date'].dt.year == 2024)]

cnd_chennai_data = may_data[may_data['Sales Team'] == 'CND Chennai']

highest_sales_cnd_chennai_may = cnd_chennai_data.groupby('Item Name')['Primary Sales'].sum().idxmax()

highest_sales_cnd_chennai_may


'NEURONZ D'

### 3. Which customer had the maximum stock returns in October for the Bangalore HQ?
---

#### Note: The data is available only up to September, so it is not possible to calculate Question 3. If you have the October data, simply uncomment `max_stock_returns_customer_oct` and run the script to obtain the October results.

In [7]:
# 3. Customer with maximum stock returns in October for Bangalore HQ

october_data = data[(data['Date'].dt.month == 10) & (data['Date'].dt.year == 2024)]

bangalore_hq_data = october_data[october_data['HQ'] == 'HQ-Bangalore']

# max_stock_returns_customer_oct = bangalore_hq_data.groupby('Customer')['Against Expiry'].sum().idxmax()

# max_stock_returns_customer_oct
bangalore_hq_data


,Sr,ID,Customer,Primary Sales,Cancelled,Is added to primary log,Against Expiry,Breakage,Claim Offer,Quantity,...,Free Item,HQ,Sales Team,Return for Reason,Sales Return,Sales Invoice Name,Rate Difference,Item Name,Free Item Value,Date


### 4. Which sales team had the maximum percentage of primary sales returned due to expiry?
---

In [24]:
# Filter data for returns due to expiry
expiry_data = data[data['Return for Reason'] == 'Expired']

# Group by 'Sales Team' and calculate the return percentage
result = (
    expiry_data.groupby('Sales Team')
    .agg({
        'Against Expiry': 'sum',
        'Primary Sales': 'sum'
    })
    .reset_index()
)

# Add a new column for return percentage
result['Return_Percentage'] = (result['Against Expiry'] / result['Primary Sales']) * 100

# Find the sales team with the maximum return percentage
max_return_team = result.loc[result['Return_Percentage'].idxmax()]

# Display the result
print(max_return_team)

Sales Team           Aura & Proxima Chennai
Against Expiry                   -305792.93
Primary Sales                    -305792.93
Return_Percentage                     100.0
Name: 0, dtype: object


### 5. What percentage of overall primary sales was affected by breakage?
---

In [25]:
# 5. Percentage of overall primary sales affected by breakage

total_primary_sales = data['Primary Sales'].sum()

total_breakage = data['Breakage'].sum()

breakage_percentage = (total_breakage / total_primary_sales) * 100
breakage_percentage

np.float64(-0.3892831591996956)

### 6. Primary sales for Delhi HQ in September
---

### Note: There is no specific `HQ-Delhi`. If you want to combine all zones in Delhi, the results are shown below.

In [26]:
# 6. Primary sales for Delhi HQ in September

delhi_hq_september_sales = september_data[september_data['Sales Team'] == 'Elbrit Delhi']['Primary Sales'].sum()
delhi_hq_september_sales


np.float64(592474.0)

### 7. Sales of "Britorva 20" for PALEPU PHARMA DIST PVT LTD under Coimbatore HQ in September
---

In [11]:

# 7. Sales of "Britorva 20" for PALEPU PHARMA DIST PVT LTD under Coimbatore HQ in September

britorva_sales_palepu_coimbatore = september_data[

    (september_data['HQ'] == 'HQ-Coimbatore') & 

    (september_data['Customer'] == 'PALEPU PHARMA DIST PVT LTD') & 

    (september_data['Item Name'] == 'Britorva 20')

]['Primary Sales'].sum()

britorva_sales_palepu_coimbatore


np.float64(0.0)

In [ ]:

# Load the dataset
file_path = 'ELB-Sales-Data.xlsx'  # Replace with your file path
sales_data = pd.read_excel(file_path, sheet_name='Elbrit Sales Log')

# Ensure the 'Date' column is in datetime format
sales_data['Date'] = pd.to_datetime(sales_data['Date'], errors='coerce')

# Filter data to retain only valid rows with 'Primary Sales' and 'Date'
sales_data = sales_data.dropna(subset=['Primary Sales', 'Date'])

# Extract month and year for aggregation
sales_data['YearMonth'] = sales_data['Date'].dt.to_period('M')

# Step 1: Aggregate monthly sales
monthly_sales = (
    sales_data.groupby('YearMonth')['Primary Sales']
    .sum()
    .reset_index()
)
monthly_sales['YearMonth'] = monthly_sales['YearMonth'].dt.to_timestamp()

# Step 2: Apply Exponential Smoothing for Forecasting
# Check if sufficient data is available
if len(monthly_sales) < 3:
    raise ValueError("Insufficient data for forecasting. At least 3 periods are required.")

# Initialize and fit the model
model = ExponentialSmoothing(
    monthly_sales['Primary Sales'], 
    trend='add', 
    seasonal=None, 
    initialization_method="estimated"
)
model_fit = model.fit()

# Forecast for the next two months (October & November)
forecast_periods = 2
forecast = model_fit.forecast(steps=forecast_periods)

# Output the results
forecast.index = pd.date_range(
    start=monthly_sales['YearMonth'].iloc[-1] + pd.offsets.MonthBegin(),
    periods=forecast_periods,
    freq='MS'
)

print("Historical Monthly Sales:")
print(monthly_sales)

print("\nForecasted Sales for October and November:")
print(forecast)


Historical Monthly Sales:
   YearMonth  Primary Sales
0 2024-04-01    43944346.66
1 2024-05-01    45896730.82
2 2024-06-01    47566563.99
3 2024-07-01    50067106.19
4 2024-08-01    41066121.82
5 2024-09-01    41616347.74

Forecasted Sales for October and November:
2024-10-01    3.989942e+07
2024-11-01    3.744539e+07
Freq: MS, dtype: float64


e:\working\venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [ ]:



# Suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Load data
file_path = 'ELB-Sales-Data.xlsx'  # Replace with actual file path
sales_data = pd.read_excel(file_path, sheet_name='Elbrit Sales Log')

# Ensure 'Date' column is in datetime format
sales_data['Date'] = pd.to_datetime(sales_data['Date'], errors='coerce')

# Drop rows with missing values in relevant columns
sales_data = sales_data.dropna(subset=['Primary Sales', 'Date'])

# Add a YearMonth column for monthly aggregation
sales_data['YearMonth'] = sales_data['Date'].dt.to_period('M')

# Forecasted Value of Primary Sales for October
monthly_sales = sales_data.groupby('YearMonth', as_index=False)['Primary Sales'].sum()

if len(monthly_sales) >= 3:  # Ensure enough data points
    model = ExponentialSmoothing(monthly_sales['Primary Sales'], trend='add', seasonal=None)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)  # Get the forecast as a Series
    forecast_primary_sales = forecast.iloc[0] if not forecast.empty else None
else:
    forecast_primary_sales = None

# Product with Highest Forecasted Sales for "CND Chennai" in November
cnd_chennai_data = sales_data[sales_data['Sales Team'].str.contains('CND Chennai', na=False)]
cnd_chennai_monthly = cnd_chennai_data.groupby(['YearMonth', 'Item Name'])['Primary Sales'].sum().unstack()

if not cnd_chennai_monthly.empty and len(cnd_chennai_monthly) >= 3:
    forecast_cnd = cnd_chennai_monthly.sum(axis=1).iloc[-3:]  # Last 3 months
    highest_forecasted_product = cnd_chennai_monthly.sum(axis=0).idxmax()
else:
    highest_forecasted_product = None

# Forecasted Sales of "Britorva 20" in October under Coimbatore HQ
britorva_data = sales_data[
    (sales_data['HQ'].str.contains('Coimbatore', case=False, na=False)) &
    (sales_data['Item Name'].str.contains('Britorva 20', case=False, na=False))
]
britorva_monthly = britorva_data.groupby('YearMonth', as_index=False)['Primary Sales'].sum()

if len(britorva_monthly) >= 3:
    try:
        # Log transformation for better stability
        britorva_monthly['Primary Sales'] = np.log1p(britorva_monthly['Primary Sales'])

        model_britorva = ExponentialSmoothing(
            britorva_monthly['Primary Sales'],
            trend='add',
            seasonal=None
        )
        # Fit the model (no maxiter argument needed)
        model_britorva_fit = model_britorva.fit()

        # Forecast and reverse log transformation
        forecast_britorva_log = model_britorva_fit.forecast(steps=1).iloc[0]
        forecast_britorva = np.expm1(forecast_britorva_log)

    except Exception as e:
        print(f"Error during forecasting: {e}")
        forecast_britorva = None

else:
    print("Insufficient data for forecasting 'Britorva 20' sales.")
    forecast_britorva = None

# Print Results
print(f"1. Forecasted Primary Sales for October: {forecast_primary_sales if forecast_primary_sales is not None else 'Not Available'}")
print(f"2. Product with Highest Forecasted Sales for 'CND Chennai' in November: {highest_forecasted_product if highest_forecasted_product is not None else 'Not Available'}")
print(f"3. Forecasted Sales of 'Britorva 20' in October under Coimbatore HQ: {forecast_britorva if forecast_britorva is not None else 'Not Available'}")


1. Forecasted Primary Sales for October: 39899423.35804366
2. Product with Highest Forecasted Sales for 'CND Chennai' in November: NEURONZ D
3. Forecasted Sales of 'Britorva 20' in October under Coimbatore HQ: 39748.982771089286
